# Bienvenidos a la 8va práctica de Física Computacional 2024-2 
* Alumna Ayudante: Valentina Santelices
    * Correo: vsantelices2021@udec.cl
* Alumno Ayudante: Juan Ortega
    * Correo: juortega2019@udec.cl

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import imageio

# Parámetros físicos y del sistema
L = 1.0       # Longitud del sistema
dx = 0.01     # Paso espacial
c = 1.0       # Velocidad de la onda

# Paso de tiempo para cumplir la condición CFL (estabilidad numérica)
dt = 0.707 * dx / c  

# Crear la malla 2D
x = np.arange(0, L + dx, dx)
y = np.arange(0, L + dx, dx)
xx, yy = np.meshgrid(x, y)  # Malla de puntos en 2D

npts = len(x)  # Número de puntos espaciales
nsteps = 100   # Número de pasos temporales

# Inicializar la matriz 3D para almacenar los estados (pasado, presente, futuro)
f = np.zeros((npts, npts, 3))

# Definir el pulso inicial como una campana gaussiana
xc, w = 0.5, 0.05  # Centro y ancho del pulso
f[:, :, 0] = np.exp(-((xx - xc)**2 + (yy - xc)**2) / w**2)  # Condición inicial

# Lista para almacenar los frames de la animación
frames = []

# Primer paso: Usar el estado inicial para calcular el estado en t + Δt
f[1:-1, 1:-1, 1] = f[1:-1, 1:-1, 0] + \
    0.5 * (c * dt / dx)**2 * (
        f[:-2, 1:-1, 0] + f[2:, 1:-1, 0] - 2 * f[1:-1, 1:-1, 0] +
        f[1:-1, :-2, 0] + f[1:-1, 2:, 0] - 2 * f[1:-1, 1:-1, 0]
    )

# Bucle principal para los siguientes pasos (Leapfrog)
for k in range(nsteps):
    # Calcular el siguiente estado en t + Δt
    f[1:-1, 1:-1, 2] = -f[1:-1, 1:-1, 0] + 2 * f[1:-1, 1:-1, 1] + \
        (c * dt / dx)**2 * (
            f[:-2, 1:-1, 1] + f[2:, 1:-1, 1] - 2 * f[1:-1, 1:-1, 1] +
            f[1:-1, :-2, 1] + f[1:-1, 2:, 1] - 2 * f[1:-1, 1:-1, 1]
        )

    # Actualizar los estados para el siguiente ciclo
    f[:, :, 0] = f[:, :, 1]  # El presente se convierte en pasado
    f[:, :, 1] = f[:, :, 2]  # El futuro se convierte en presente

    # Guardar los frames cada 10 pasos
    if k % 10 == 0:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.plot_surface(xx, yy, f[:, :, 2], cmap=cm.coolwarm)
        ax.set_title(f'Tiempo: {k * dt:.2f} s')
        ax.set_zlim(-0.25, 1.0)

        # Guardar el frame en la lista de imágenes
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(image)
        plt.close(fig)

# Crear el GIF a partir de los frames
gif_path = "onda_2d.gif"
imageio.mimsave(gif_path, frames, fps=5)

gif_path


/tmp/ipykernel_15894/1663524650.py:62: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')


'onda_2d.gif'